# Match Transitivity
As input, we use the symmetric dataset

Here, I programmed an attempt to enforce transitivity with 4 matches or 3.
 * If four matches, we check the tree matched lines and we add the missing matches
 * After, if three matches, we do the same 

Unfortunately there are tricky cases and we would need a kind of Hungarian algorithm to solve it.

Pierre Nugues

In [65]:
import pandas as pd
import json
from tqdm import tqdm

In [66]:
# pd.set_option('display.max_colwidth', None)

In [67]:
"""df = pd.read_json(
    "hf://datasets/albinandersson/Nordisk-Familjebok-Headword-Classified-Matched-Linked/data.jsonl", lines=True)
df.to_json('~/nf.json')
    """

'df = pd.read_json(\n    "hf://datasets/albinandersson/Nordisk-Familjebok-Headword-Classified-Matched-Linked/data.jsonl", lines=True)\ndf.to_json(\'~/nf.json\')\n    '

In [68]:
df = pd.read_json('nf_sym.json')

In [69]:
df.head(10)

,entry_id,headword,definition,type,edition,E1_match,E2_match,E3_match,E4_match,QID
0,E1_1,A,A är den första bokstafven i alla indoeuropeis...,0,E1,,,,,
1,E1_2,A,"A, Lat. prepos. Se Ab.",0,E1,,,,,
2,E1_3,Aa,"Aa ( utt. a ; Ach l. Aach, af Fornt. aha, vatt...",0,E1,,,,,
3,E1_4,Aabenraa,"Aabenraa ( Åbenrå ; på tyska Apenrade ), stad ...",1,E1,,E2_7,E3_9,,
4,E1_5,Aachen,"Aachen ( utt. ak - ; Lat. Aquisgranum, Fr. [ r...",1,E1,,E2_10,E3_13,,
5,E1_6,Aafjord,"Aafjord ( Åfjord ), stad i Norge, Trondhjems [...",1,E1,,,,,
6,E1_7,Aagesön,"Aagesön [ ågesön ], Svend ( Lat. Sveno Agonis ...",2,E1,,E2_16,,,
7,E1_8,Aahausen,"Aahausen [ a - ] l. Anhausen, by i Würtemberg....",1,E1,,,,,
8,E1_9,Aak,"Aak [ ak ], Holl., ett slags lastpråm eller fl...",0,E1,,,,,
9,E1_10,Aakirkeby,"Aakirkeby ( Åkirkeby ), köping på Bornholm. 74...",1,E1,,E2_18,E3_18,,


## Example

In [70]:
df[df['headword'] == 'Aagaard']

,entry_id,headword,definition,type,edition,E1_match,E2_match,E3_match,E4_match,QID
85630,E1_85631,Aagaard,"Aagaard [ ågård ], K. F., dansk målare. Se Ågård.",2,E1,,E2_14,,E4_15,
117486,E2_14,Aagaard,"Aagaard [ ågår ]. Karl Frederik, dansk målare,...",2,E2,E1_85631,,E3_14,E4_15,
302549,E3_14,Aagaard,"Aagaard [ ågår ], Carl Frederik, dansk målare ...",2,E3,,E2_14,,E4_15,
329014,E4_15,Aagaard,"Aagaard [ ågår ], Carl Frederik, dansk målare ...",2,E4,E1_85631,E2_14,E3_14,,
329015,E4_16,Aagaard,"Aagaard [ ägård ], Bjarne, norsk affärsman ( f...",2,E4,,,,,


Another example that does not go to solve

In [ ]:
{'E2_160182', 'E2_165518', 'E2_50422', 'E2_52287', 'E2_53948'}

In [72]:
df[df['entry_id'] == 'E2_160182']

,entry_id,headword,definition,type,edition,E1_match,E2_match,E3_match,E4_match,QID
277654,E2_160182,Frösunda,"Frösunda, Stora, egendom i Solna socken, Stock...",1,E2,,,,E4_30164,


In [73]:
df[df['entry_id'] == 'E4_30164']

,entry_id,headword,definition,type,edition,E1_match,E2_match,E3_match,E4_match,QID
359163,E4_30164,Frösunda,"Frösunda. 1 ) Socken i s. Uppland, Seming - hu...",1,E4,E1_23954,E2_160182,E3_8384,,


In [77]:
df[df['entry_id'] == 'E1_23954']

,entry_id,headword,definition,type,edition,E1_match,E2_match,E3_match,E4_match,QID
23953,E1_23954,Frösunda,"Frösunda, socken i Stockholms län, Seminghundr...",1,E1,,E2_41636,E3_8384,E4_30164,


In [78]:
df[df['entry_id'] == 'E3_8384']

,entry_id,headword,definition,type,edition,E1_match,E2_match,E3_match,E4_match,QID
310919,E3_8384,Frösunda,"Frösunda. 1. Socken i s. Uppland, Seming - hun...",1,E3,E1_23954,E2_41636,,E4_30164,


In [79]:
df[df['entry_id'] == 'E2_41636']

,entry_id,headword,definition,type,edition,E1_match,E2_match,E3_match,E4_match,QID
159108,E2_41636,Frösunda,"Frösunda, socken i Stockholms län, Seminghundr...",1,E2,E1_23954,,E3_8384,,


In [80]:
df[df['entry_id'] == 'E2_160182']

,entry_id,headword,definition,type,edition,E1_match,E2_match,E3_match,E4_match,QID
277654,E2_160182,Frösunda,"Frösunda, Stora, egendom i Solna socken, Stock...",1,E2,,,,E4_30164,


We have the links:
 * E2_41636 <-> E1_23954
 * E2_41636 <-> E3_8384
 * E1_23954 <-> E4_30164
 * E1_23954 <-> E3_8384
 * E3_8384 <-> E4_30164 <-> E2_160182

## Four Matches

Attempt to write a program to establish transitivity from E2 --> E1, E3, E4

In [87]:
df.loc[(df['edition'] == 'E2') &
       (df['E1_match']) &
       (df['E3_match']) &
       (df['E4_match'])]

,entry_id,headword,definition,type,edition,E1_match,E2_match,E3_match,E4_match,QID
117486,E2_14,Aagaard,"Aagaard [ ågår ]. Karl Frederik, dansk målare,...",2,E2,E1_85631,,E3_14,E4_15,
117487,E2_15,Aagesen,"Aagesen [ åg - ], Andreas, dansk rättslärd, f....",2,E2,E1_85632,,E3_15,E4_18,
117492,E2_20,Aalberg,"Aalberg [ al - ], Ida Emilia, finsk skådespele...",2,E2,E1_85635,,E3_22,E4_22,
117502,E2_30,Aalst,"Aalst l. Aelst [ al - ], fr. Alost, stad i bel...",1,E2,E1_22,,E3_37,E4_27,
117511,E2_39,Aargau,"Aargau [ ar - ], kanton i schweiz. statsförbun...",1,E2,E1_29,,E3_46,E4_39,
...,...,...,...,...,...,...,...,...,...,...
192789,E2_75317,Lillesand,"Lillesand, lastageplats i Nedenes amt, Norge, ...",1,E2,E1_43890,,E3_26219,E4_50943,
193813,E2_76341,Llandudno,"Llandudno [ ländaMnåu ], engelsk badort i norr...",1,E2,E1_44435,,E3_26348,E4_51659,
194815,E2_77343,Lovisa,"Lovisa, svenska drottningar :",2,E2,E1_44898,,E3_26413,E4_52033,
276832,E2_159360,Flint,"Flint, stad i Michigan, Förenta staterna, vid ...",1,E2,E1_99899,,E3_6348,E4_28165,


In [82]:
idx_1 = df.loc[(df['edition'] == 'E1') &
               (df['E2_match']) &
               (df['E3_match']) &
               (df['E4_match'])].index

In [83]:
idx_2 = df.loc[(df['edition'] == 'E2') &
               (df['E1_match']) &
               (df['E3_match']) &
               (df['E4_match'])].index

In [84]:
idx_3 = df.loc[(df['edition'] == 'E3') &
               (df['E1_match']) &
               (df['E2_match']) &
               (df['E4_match'])].index

In [85]:
idx_4 = df.loc[(df['edition'] == 'E4') &
               (df['E1_match']) &
               (df['E2_match']) &
               (df['E3_match'])].index

In [86]:
df.iloc[117486][['E1_match', 'E3_match', 'E4_match']]

E1_match    E1_85631
E3_match       E3_14
E4_match       E4_15
Name: 117486, dtype: object

In [ ]:
df[df['entry_id'] == 'E1_85631']

In [ ]:
def missing_matches(df, idx, ed):
    all_eds = {'E1', 'E2', 'E3', 'E4'}
    compl_eds = all_eds.difference({ed})
    matching_eds = [ed + '_match' for ed in compl_eds]
    matching_entries = df.iloc[idx][matching_eds].to_list()
    matching_pairs = list(zip(matching_eds, matching_entries))
    for m in matching_pairs:
        class_members = set(matching_pairs).difference({m})
        for class_member in class_members:
            target_idx = df[df['entry_id'] == m[1]].index
            if not df.loc[target_idx, class_member[0]].any():
                df.loc[target_idx, class_member[0]] = class_member[1]
    return

In [ ]:
missing_matches(df, 117486, 'E2')

In [ ]:
df[df['entry_id'] == 'E1_85631']

In [ ]:
for idx in tqdm(idx_1):
    missing_matches(df, idx, 'E1')

In [ ]:
for idx in tqdm(idx_2):
    missing_matches(df, idx, 'E2')

In [ ]:
for idx in tqdm(idx_3):
    missing_matches(df, idx, 'E3')

In [ ]:
for idx in tqdm(idx_4):
    missing_matches(df, idx, 'E4')

In [ ]:
df.loc[(df['edition'] == 'E1') &
       (df['E2_match']) &
       (df['E3_match']) &
       (df['E4_match'])]

In [ ]:
df.loc[(df['edition'] == 'E2') &
       (df['E1_match']) &
       (df['E3_match']) &
       (df['E4_match'])]

In [ ]:
df.loc[(df['edition'] == 'E3') &
       (df['E1_match']) &
       (df['E2_match']) &
       (df['E4_match'])]

In [ ]:
df.loc[(df['edition'] == 'E4') &
       (df['E1_match']) &
       (df['E2_match']) &
       (df['E3_match'])]

In [ ]:
l1_2 = df.loc[(df['edition'] == 'E1') &
       (df['E2_match']) &
       (df['E3_match']) &
       (df['E4_match'])]['E2_match'].tolist()
l1_2 = sorted(l1_2)
len(set(l1_2))

In [ ]:
l2 = df.loc[(df['edition'] == 'E2') &
       (df['E1_match']) &
       (df['E3_match']) &
       (df['E4_match'])]['entry_id'].tolist()
l2 = sorted(l2)
len(set(l2))

In [ ]:
set(l1_2) ^ set(l2)

In [ ]:
df[df['entry_id'] == 'E1_23954']

In [ ]:
df[df['entry_id'] == 'E2_160182']

In [ ]:
#df.to_json('nf_sym_trans.json')

First failed attempt to write the symmetry program

In [ ]:
"""
def set_symmetry_old(df, ed1, ed2):
    concord_m, discord_m, no_m = 0, 0, 0
    matching_ed1 = ed1 + '_match'
    matching_ed2 = ed2 + '_match'
    for i in tqdm(range(len(df))):
        if df.iloc[i]['edition'] != ed1:
            continue
        entry = df.iloc[i]['entry_id']
        matching_entry = df.iloc[i][matching_ed2]
        if matching_entry:  # We have a matching entry
            # print(i, df.iloc[i]['entry_id'], matching_entry)
            if df[(df['edition'] == ed2) &
                  (df['entry_id'] == matching_entry) &
                    (df[matching_ed1] == entry)].any().any():  # Case 1: We have a symmetry
                concord_m += 1
                # print(df[df['entry_id'] == matching_entry])
            elif df[(df['edition'] == ed2) &
                    (df['entry_id'] == matching_entry) &
                    (df[matching_ed1] == '')].any().any():  # Case 2: We have a unique relation
                no_m += 1
                df.loc[df[df['entry_id'] == matching_entry].index,
                       matching_ed1] = entry
                # print(i, df.iloc[i]['entry_id'], matching_entry)
                # print(df[df['entry_id'] == matching_entry].index)
                # break
            else:  # Case 3: We have two nonsymmetrical relations. We remove them.
                # This will create one-way relations.
                # We need to run again the loop (twp-pass procedure)
                discord_m += 1
                df.loc[df[df['entry_id'] == entry].index,
                       matching_ed2] = ''
                df.loc[df[df['entry_id'] == matching_entry].index,
                       matching_ed1] = ''
                # print(i, df.iloc[i]['entry_id'], matching_entry)
                # print(df[df['entry_id'] == matching_entry].index)
                # break
    return concord_m, discord_m, no_m
"""